# SQL Server -> Pandas -> Excel

Import neophodnih biblioteka. Sve bi trebalo da vec imas instalirano, ali ako dobijes gresku tipa `No module named 'pyodbc'` prilikom izvrsenja sljedece linije - modul mozes instalirati komandom `pip install pyodbc` (sa komandne linije).

In [1]:
import pandas as pd
import pyodbc

### Prikupljanje podataka

Kreiranje konekcije (nesto nalik VB-u). Ovdje koristim `Integrated Security`, a ukoliko koristis `SQL Authentication` - mozes umjesto `Trusted_Connection=yes` staviti `Trusted_Connection=no;Uid=username;Pwd=password`

In [2]:
conn = pyodbc.connect("DRIVER={ODBC Driver 13 for SQL Server};SERVER=localhost;DATABASE=Northwind;Trusted_Connection=yes")

Kreiranje odgovarajucih SQL upita (mozes tu unijeti sto god hoces). Ja kreiram dva upita sa razlicim podacima kako bi ih najzad eksportovao u Excel (dva odvojena Sheet-a)

In [3]:
sql1 = "EXEC [dbo].[Ten Most Expensive Products]"
sql2 = "SELECT TOP 20 * FROM Products"

Izvrsavanje SQL upita i smjestanja rezultata u `Pandas DataFrame` objekte `df1` i `df2`

In [4]:
df1 = pd.read_sql(sql1, conn)
df2 = pd.read_sql(sql2, conn)

Ovako izgledaju podaci

In [5]:
df1.head()

,TenMostExpensiveProducts,UnitPrice
0,Côte de Blaye,263.50
1,Thüringer Rostbratwurst,123.79
2,Mishi Kobe Niku,97.00
3,Sir Rodney's Marmalade,81.00
4,Carnarvon Tigers,62.50


In [6]:
df2.head()

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,False
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,False
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,False
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,False
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,True


Zatvaranje konekcije

In [7]:
conn.close()

### Eksport podataka u Excel Sheet

Ukoliko bi eksport-ovali jedan `DataFrame` u Excel, mogli bi koristiti sljedecu komandu:

`df.to_excel("podaci.xlsx", sheet_name="Artikli", index=False)`

Medjutim, s' obzirom da trebamo eksport-ovati vise `DataFrame`-ova, koristimo `ExcelWriter` na sljedeci nacin:

In [8]:
with pd.ExcelWriter("podaci.xlsx") as writer:
    df1.to_excel(writer, sheet_name="Tabela 1", index=False)
    df2.to_excel(writer, sheet_name="Tabela 2", index=False)

To je otprilike to. Imas na raspolaganju razne opcije eksport-ovanja u Excel. Mozes pogledati Help.

In [9]:
help(df1.to_excel)

Help on method to_excel in module pandas.core.generic:

to_excel(excel_writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True, freeze_panes=None) -> None method of pandas.core.frame.DataFrame instance
    Write object to an Excel sheet.
    
    To write a single object to an Excel .xlsx file it is only necessary to
    specify a target file name. To write to multiple sheets it is necessary to
    create an `ExcelWriter` object with a target file name, and specify a sheet
    in the file to write to.
    
    Multiple sheets may be written to by specifying unique `sheet_name`.
    With all data written to the file it is necessary to save the changes.
    Note that creating an `ExcelWriter` object with a file name that already
    exists will result in the contents of the existing file being erased.
    
    Parameters
    -------

### Slanje podataka na E-mail

Evo ti jedne funkcije koju mozes koristiti za slanje e-mail poruka sa attachment-om, slikama i sl. Funkcionise sasvim OK sa raznim e-mail provajderima i tu ne bi trebalo da imas problem.

In [4]:
import smtplib, os
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.utils import COMMASPACE, formatdate
from email import encoders

# Ti zamijeni default vrijednosti tvojim, ili ih zadaj tokom pozivanja funkcije
def send_mail(send_from, send_to, subject, text, files=None, data_attachments=None, 
              server="smtp.office365.com", port=587, tls=True, html=True, images=None, username=None, password=None):

    if files is None: files = []
    if images is None: images = []
    if data_attachments is None: data_attachments = []

    msg = MIMEMultipart('related')
    msg['From'] = send_from
    msg['To'] = send_to if isinstance(send_to, str) else COMMASPACE.join(send_to)
    msg['Date'] = formatdate(localtime=True)
    msg['Subject'] = subject

    msg.attach( MIMEText(text, 'html' if html else 'plain') )

    for f in files:
        part = MIMEBase('application', "octet-stream")
        part.set_payload( open(f,"rb").read() )
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', 'attachment; filename="%s"' % os.path.basename(f))
        msg.attach(part)

    for f in data_attachments:
        part = MIMEBase('application', "octet-stream")
        part.set_payload( f['data'] )
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', 'attachment; filename="%s"' % f['filename'])
        msg.attach(part)

    for (n, i) in enumerate(images):
        fp = open(i, 'rb')
        msgImage = MIMEImage(fp.read())
        fp.close()
        msgImage.add_header('Content-ID', '<image{0}>'.format(str(n+1)))
        msg.attach(msgImage)

    smtp = smtplib.SMTP(server, int(port))
    if tls:
        smtp.starttls()

    if username is not None:
        smtp.login(username, password)
    smtp.sendmail(send_from, send_to, msg.as_string())
    smtp.close()

I, najzad, mozes pozvati funkciju i poslati e-mail sa generisanim Excel prilogom na sljedeci nacin

In [5]:
send_mail('"Djordje NAJDANOVIC" <djordje@azalea-maritime.com>', 
          ['djordje@najdanovic.com'], 
          'E-mail subject', 
          '<h1>Naslov e-mail poruke</h1><p>Paragraf</p>', 
          files=['podaci.xlsx'], 
          username='djordje@azalea-maritime.com', 
          password='xxxxxxxxxxxxx')